# Análise Exploratória - Dados INMET 2025

**Objetivo:** Analisar dados climáticos do INMET para identificar padrões relevantes para previsão de dengue

**Dados:** Arquivos CSV do INMET com informações de múltiplas estações meteorológicas

**Foco:** Precipitação, temperatura e umidade - variáveis chave para proliferação do *Aedes aegypti*

## 1. Setup e Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✅ Bibliotecas carregadas com sucesso!")

## 2. Carregar Dados Processados

In [ ]:
# Carregar dados SINAN (dengue) que acabamos de baixar
sinan_path = Path('../data/raw/sinan/dengue_2025.parquet')

if sinan_path.exists():
    df_sinan = pd.read_parquet(sinan_path)
    print(f"✅ Dados SINAN carregados: {len(df_sinan):,} registros")
    print(f"\nColunas disponíveis: {list(df_sinan.columns)}")
    print(f"\nPrimeiras linhas:")
    print(df_sinan.head())
else:
    print("❌ Arquivo SINAN não encontrado!")

# Verificar se existem dados INMET processados
inmet_path = Path('../data/processed/inmet/inmet_consolidated_2025.parquet')
if inmet_path.exists():
    df_inmet = pd.read_parquet(inmet_path)
    print(f"\n✅ Dados INMET carregados: {len(df_inmet):,} registros")
    print(f"Colunas INMET: {list(df_inmet.columns)}")
else:
    print("\n❌ Dados INMET não processados ainda. Vamos trabalhar apenas com dados SINAN por enquanto.")

## 3. Visão Geral dos Dados

In [ ]:
# Análise dos dados SINAN (Dengue)
print("="*70)
print("RESUMO DOS DADOS SINAN - DENGUE 2025")
print("="*70)
print(f"Total de registros: {len(df_sinan):,}")

# Verificar colunas importantes
print(f"\nColunas disponíveis:")
for col in df_sinan.columns:
    print(f"  - {col}")

# Análise de dados faltantes
print(f"\nDados faltantes por coluna:")
missing_data = df_sinan.isnull().sum()
for col, missing in missing_data.items():
    if missing > 0:
        print(f"  - {col}: {missing:,} ({missing/len(df_sinan)*100:.1f}%)")

# Informações sobre datas (se existir coluna de data)
date_cols = [col for col in df_sinan.columns if 'data' in col.lower() or 'dt_' in col.lower()]
if date_cols:
    print(f"\nColunas de data encontradas: {date_cols}")
    for col in date_cols:
        print(f"  - {col}: {df_sinan[col].dtype}")
        if df_sinan[col].dtype == 'object':
            print(f"    Exemplos: {df_sinan[col].dropna().head(3).tolist()}")

In [ ]:
# Primeiras linhas
df.head(10)

In [ ]:
# Informações de tipos e missings
df.info()

In [ ]:
# Visualizar dados de dengue
print("Primeiras 10 linhas dos dados de dengue:")
df_sinan.head(10)


In [ ]:
# Análise de distribuição geográfica dos casos de dengue
print("="*70)
print("DISTRIBUIÇÃO GEOGRÁFICA DOS CASOS DE DENGUE")
print("="*70)

# Verificar se há colunas de localização
location_cols = [col for col in df_sinan.columns if any(term in col.lower() for term in ['uf', 'estado', 'municipio', 'cidade', 'regiao'])]
print(f"Colunas de localização encontradas: {location_cols}")

if location_cols:
    for col in location_cols:
        print(f"\nDistribuição por {col}:")
        value_counts = df_sinan[col].value_counts()
        print(value_counts.head(10))
        
        # Criar gráfico de barras
        plt.figure(figsize=(12, 6))
        value_counts.head(15).plot(kind='bar')
        plt.title(f'Distribuição de casos por {col}')
        plt.xlabel(col)
        plt.ylabel('Número de casos')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
else:
    print("Nenhuma coluna de localização encontrada.")


In [ ]:
# Análise temporal dos casos de dengue
print("="*70)
print("ANÁLISE TEMPORAL DOS CASOS DE DENGUE")
print("="*70)

# Procurar colunas de data
date_cols = [col for col in df_sinan.columns if 'data' in col.lower() or 'dt_' in col.lower()]
print(f"Colunas de data encontradas: {date_cols}")

if date_cols:
    for col in date_cols:
        print(f"\nAnálise da coluna {col}:")
        print(f"Tipo de dados: {df_sinan[col].dtype}")
        print(f"Valores únicos: {df_sinan[col].nunique()}")
        print(f"Valores nulos: {df_sinan[col].isnull().sum()}")
        
        # Mostrar exemplos de valores
        sample_values = df_sinan[col].dropna().head(5).tolist()
        print(f"Exemplos de valores: {sample_values}")
        
        # Tentar converter para datetime se possível
        try:
            if df_sinan[col].dtype == 'object':
                # Tentar diferentes formatos de data
                df_sinan[f'{col}_parsed'] = pd.to_datetime(df_sinan[col], errors='coerce')
                valid_dates = df_sinan[f'{col}_parsed'].dropna()
                if len(valid_dates) > 0:
                    print(f"Datas válidas encontradas: {len(valid_dates)}")
                    print(f"Período: {valid_dates.min()} a {valid_dates.max()}")
                    
                    # Análise mensal
                    df_sinan['mes'] = df_sinan[f'{col}_parsed'].dt.month
                    df_sinan['ano'] = df_sinan[f'{col}_parsed'].dt.year
                    
                    # Contar casos por mês
                    casos_por_mes = df_sinan.groupby(['ano', 'mes']).size().reset_index(name='casos')
                    print(f"\nCasos por mês:")
                    print(casos_por_mes)
                    
                    # Gráfico temporal
                    plt.figure(figsize=(15, 6))
                    casos_por_mes.set_index(['ano', 'mes']).plot(kind='bar')
                    plt.title(f'Casos de dengue por mês - {col}')
                    plt.xlabel('Ano-Mês')
                    plt.ylabel('Número de casos')
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    plt.show()
                    
        except Exception as e:
            print(f"Erro ao processar datas: {e}")
else:
    print("Nenhuma coluna de data encontrada.")


## 4. Análise de Precipitação

**Por que é importante?**
- Chuvas criam criadouros para larvas do mosquito
- Picos de chuva geralmente precedem picos de dengue em 3-8 semanas
- Precipitação acumulada é um dos melhores preditores

In [ ]:
# TODO: Análise de precipitação
# - Séries temporais por estado
# - Distribuições
# - Padrões sazonais
pass

## 5. Análise de Temperatura

**Por que é importante?**
- Temperatura entre 20-30°C é ideal para o mosquito
- Afeta taxa de reprodução e tempo de incubação viral
- Temperaturas extremas (muito frias ou quentes) reduzem atividade

In [ ]:
# TODO: Análise de temperatura
pass

## 6. Análise de Umidade

**Por que é importante?**
- Umidade alta aumenta sobrevivência do mosquito
- Afeta longevidade e capacidade de transmissão
- Combinação temperatura + umidade é crítica

In [ ]:
# TODO: Análise de umidade
pass

## 7. Padrões Sazonais

Identificar períodos de maior risco para dengue baseado em clima

In [ ]:
# TODO: Análise sazonal
# - Agregação mensal
# - Comparação entre estados
# - Identificação de "estações de risco"
pass

## 8. Agregação Semanal

Preparar dados para junção com séries temporais de dengue (que são semanais)

In [ ]:
# TODO: Agregar por semana epidemiológica
pass

## 9. Exportar Dados para Modelagem

In [ ]:
# TODO: Salvar datasets agregados
# - Por estado + semana
# - Por estação + semana
# - Features derivadas (lags, médias móveis)
pass

## 10. Conclusões e Próximos Passos

**Principais achados:**
- (A ser preenchido após análise)

**Próximos passos:**
1. Carregar dados de dengue do SINAN
2. Fazer join espacial (estação INMET → município mais próximo)
3. Calcular correlações cruzadas (climate lag vs casos dengue)
4. Treinar modelos preditivos